# Ajuste do circuito RLC com R = 0Ω

In [44]:
import pandas as pd
import altair as alt

### Leitura dos dados de tensão obtidos através do osciloscópio

In [45]:
p4a_data = pd.read_csv('./p4a_data0.csv')
p4a_data.head()

,Time(s),CH1V,CH2V
0,-0.002000,-0.082667,1.197600
1,-0.001998,-0.033867,1.515733
2,-0.001996,-0.056000,1.696800
3,-0.001994,-0.016000,1.838133
4,-0.001992,-0.002667,1.816000


Setando o tempo 0:

In [46]:
p4a_data["Time(s)"] = p4a_data["Time(s)"] + 2.000000e-03
p4a_data.head()

,Time(s),CH1V,CH2V
0,0.000000,-0.082667,1.197600
1,0.000002,-0.033867,1.515733
2,0.000004,-0.056000,1.696800
3,0.000006,-0.016000,1.838133
4,0.000008,-0.002667,1.816000


### Visualização inicial dos dados

In [112]:
p4a_CH1V = alt.Chart(p4a_data).mark_line(color='orange').encode(
    x = "Time(s)",
    y = "CH1V",
    tooltip = "Time(s)"
).interactive()

p4a_CH2V = alt.Chart(p4a_data).mark_point().encode(
    x = alt.X("Time(s)", title="Tempo (s)"),
    y = alt.Y("CH2V", title="Tensão (V)"),
    tooltip = "Time(s)"
).properties(title='Tensões medidas').interactive()

p4a_chart = p4a_CH2V + p4a_CH1V
p4a_chart

alt.LayerChart(...)

### Selecionando uma oscilação

In [107]:
p4a = p4a_data[(p4a_data["Time(s)"] >= 0.000978) & (p4a_data["Time(s)"] <= 0.001484)]

p4a_chart = alt.Chart(p4a).mark_point().encode(
    x = alt.X("Time(s)", scale=alt.Scale(domain=[0.000978, 0.001484])),
    y = alt.Y("CH2V"),
    tooltip = ["Time(s)", "CH2V"]
).interactive()

p4a_chart

alt.Chart(...)

### Configurando o fit para o ajuste 

In [50]:
import numpy as np
from scipy.optimize import curve_fit

# Função do modelo para o ajuste
def oscilacao_amortecida(t, Voff, A, alpha, wp, t0):
    return Voff + A * np.exp(-alpha * (t - t0)) * np.cos(wp * (t - t0))

### Ajustando a oscilação selecionada

In [111]:
# Dados experimentais
tempo = p4a['Time(s)'].to_numpy()
tensao_ruidosa = p4a['CH2V'].to_numpy()

# Ajuste dos parâmetros usando curve_fit
p0 = [0, 2.5, 10460, 10447, 0.000978]  # Estimativas iniciais para os parâmetros
params, covariance = curve_fit(oscilacao_amortecida, tempo, tensao_ruidosa, p0=p0)

# Parâmetros ajustados
Voff, A, alpha, wp, t0 = params
param_incertezas = np.sqrt(np.diag(covariance))
print(f"Parâmetros ajustados:")
print(f"Voff = {Voff:.4f} V | Incerteza: {param_incertezas[0]}")
print(f"A = {A:.4f} V | Incerteza: {param_incertezas[1]}")
print(f"alpha = {alpha:.4f} s^-1 | Incerteza: {param_incertezas[2]}")
print(f"wp = {wp:.4f} rad/s | Incerteza: {param_incertezas[3]}")
print(f"t0 = {t0:.4f} s | Incerteza: {param_incertezas[4]}")

# Calcular a frequência natural (omega_0) a partir de wp e alpha
omega_0 = np.sqrt(wp**2 + alpha**2)
print(f"w0 = {omega_0:.4f} rad/s")

# Gerar dados ajustados para comparação
tensao_ajustada = oscilacao_amortecida(tempo, *params)

# Criar dataframe para visualização
df = pd.DataFrame({
    'Tempo (s)': tempo,
    'Tensão Medida (V)': tensao_ruidosa,
    'Tensão Ajustada (V)': tensao_ajustada,
    'nome_da_curva_1': 'Dados experimentais',
    'nome_da_curva_2': 'Curva ajustada'
})

# Visualização com Altair
chart = alt.Chart(df).mark_point().encode(
    x='Tempo (s)',
    y=alt.Y('Tensão Medida (V)', title='Tensão (V)', axis=alt.Axis(grid=True)),
    tooltip=['Tempo (s)', 'Tensão Medida (V)'],
).properties(title='Ajuste de oscilação sub-amortecida').interactive()

ajuste_chart = alt.Chart(df).mark_line(color='red').encode(
    x='Tempo (s)',
    y='Tensão Ajustada (V)',
    color=alt.Color('nome_da_curva_2', legend=alt.Legend(title="", orient="top-right"), scale=alt.Scale(range=['red']))
)

ajuste_chart + chart

Parâmetros ajustados:
Voff = 0.0029 V | Incerteza: 0.0015797639400047154
A = 4.4796 V | Incerteza: 0.01300863086298964
alpha = 16720.9783 s^-1 | Incerteza: 66.00059777162474
wp = 115565.5600 rad/s | Incerteza: 71.74440096789209
t0 = 0.0010 s | Incerteza: 2.711247406313505e-08
w0 = 116768.9590 rad/s


alt.LayerChart(...)